In [1]:
filename="/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/LVO_CTP_19_20_occlusions_with_edema.xlsx"

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
import pandas as pd
raw_df=pd.read_excel(io=filename)

raw_df.shape

(119, 52)

In [3]:
#Only anterior circulation occlusions
raw_df = raw_df[raw_df['ant.circ.occl']>0]
raw_df.shape

(97, 52)

In [4]:
reports = raw_df[["fu_CT1_report", "fu_CT2_report", "fu_CT3_report", "fu_CT4_report"]]
reports

,fu_CT1_report,fu_CT2_report,fu_CT3_report,fu_CT4_report
0,NaN,NaN,NaN,NaN
1,New hematoma centered at the right insula and ...,large area of hypoattenuation within the right...,Continued evolution of right MCA territory and...,NaN
5,Evolving right middle cerebral artery infarct ...,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
...,...,...,...,...
114,NaN,NaN,NaN,NaN
115,There is no acute intracranial hemorrhage. Th...,NaN,NaN,NaN
116,There is widespread loss of gray-white matter ...,there is hypoattenuation of the left cerebral ...,There is an evolving left middle cerebral arte...,NaN
117,There is an evolving left frontal lobe infarct...,NaN,NaN,NaN


In [5]:
import networkx as nx
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData

In [6]:
#pyConTextNLP uses yml files to store entities and their regex pattern information
#This function is to be used for quick lookup of an entity for short term use
#Or to add new entities to a yml file

#"/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/modifiers.yml"
#"/mnt/c/Users/zohai/Documents/Research/Neuro_ML/Sample_Data/targets.yml"

class pyConText_itemData_initializer:
    
    def __init__(self, modifiers_path=None, targets_path=None):
        if modifiers_path is None:
            self.modifiers_path = "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml"
        else:
            self.modifiers_path = modifiers_path
            
        self.modifiers = itemData.get_items(self.modifiers_path)
        self.targets = []
            
        if targets_path is not None:
            self.targets_path = targets_path
            self.targets = itemData.get_items(self.targets_path)
    
    def add_target(self,entity,category='', regex='', direction=''):
        self.targets.append(itemData.contextItem([entity,category,regex,direction]))
        
        #Code from pyConTextNLP itemData.py
        #def get_items(_file): context_items =  [contextItem((d["Lex"], d["Type"],r"%s"%d["Regex"],d["Direction"])) for d in yaml.load_all(f0)]
    
    def add_entity_to_yml(self, yml_path, entity, category="''", regex="''", direction="''"):
        yml_entity = "Direction: "+ direction + "\n" + \
        "Lex: "+ entity + "\n" + \
        "Regex: "+ regex + "\n" + \
        "Type: " + category + "\n" + \
        "---\n" 
        
        with open(yml_path, "r+") as f:
            old = f.read()
            f.seek(0)
            #Prepending
            f.write(yml_entity + old)
            
        f.close()
        
    def get_itemData(self):
        return self.modifiers,self.targets
        


In [7]:
#Function to determine existence or rule out given entity
#Ex. Midline shift/No midline shift
#Ex. Occlusion/No occlusion

#Input is string entity that you want to rule in/out
#***Since words have DIFFERENT forms, consider using a PREFIX INSTEAD
#Ex. Entity: occlusion; sentence contains 'occluded thrombus'
# Using the prefix 'occlu' would capture both in a rudimentary fashion

class modifier_target_finder:
    def __init__(self, sentence, mod_itemData, targ_itemData):
        self.modifiers = mod_itemData
        self.targets = targ_itemData
        self.sentence = sentence
        
        self.markup = pyConText.ConTextMarkup()
        self.markup.setRawText(sentence.lower())
        self.markup.cleanText()
        
        self.markup.markItems(self.modifiers,mode="modifier")
        self.markup.markItems(self.targets,mode="target")
        self.markup.applyModifiers()
    
    #New addition
    def log(self):
        print(self.markup.__str__())
        
    
    def get_markup(self):
        return list(self.markup.edges())
            
    def find_mod_target_pair(self,desired_modifier,modifier_attrib,desired_target,target_attrib):
        #set removes duplicates
        return list(set([(modifier.getLiteral(),target.getLiteral()) 
                for (modifier,target) in self.markup.edges() 
                # getattr() will run modifier/target . getLiteral()/categoryString()/getTagID()/getConTextCategory()
                if (desired_modifier in getattr(modifier,modifier_attrib)() 
                    and desired_target in getattr(target,target_attrib)())]))
    
    
    #Returns 0 if negated
    #Returns 1 if present and not negated
    #Returns 2 if confirmed existence
    #Returns None if not in sentence
    def confirm_existence(self, entity, entity_type):
        entities = []
        for modifier,target in self.markup.edges():
            etype_func = {"getLiteral":[modifier.getLiteral,target.getLiteral],
                          "categoryString":[modifier.categoryString,target.categoryString]}
            for func in etype_func[entity_type]:
                entities.append(func())
        if entity in entities:
            neg_pairs_list = self.find_mod_target_pair('negated_existence','categoryString',entity,entity_type)
            if neg_pairs_list:
                return 0
            def_exis_pairs_list = self.find_mod_target_pair(('definite_existence'),
                                                    'categoryString', entity,entity_type)
            prob_exis_pairs_list = self.find_mod_target_pair(('definite_existence'),
                                                            'categoryString', entity,entity_type)
            if def_exis_pairs_list or prob_exis_pairs_list:
                return 2
            return 1
        else:
            return None

In [8]:
mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/modifiers_fu.yml"
targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/targets_fu.yml"
itemData_init = pyConText_itemData_initializer(mod_path,targ_path)
itemData_init.modifiers[3]

/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pyConTextNLP/itemData.py:40: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  context_items =  [contextItem((d["Lex"],


literal<<\br\b>>; category<<['right_side']>>; re<<\br\b>>; rule<<forward>>

In [9]:
mtf = modifier_target_finder("There is evolution of large left anterior and middle cerebral artery territory infarcts with loss of gray-white matter differentiation and loss of the sulci",
                             itemData_init.get_itemData()[0],itemData_init.get_itemData()[1])
mtf.get_markup()

[(<id> 216352204347299563531621281619235742488 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 216377571620373380687231982521254322968 </id> <phrase> territory </phrase> <category> ['brain_anatomy'] </category> ),
 (<id> 216352204347299563531621281619235742488 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 216377180233250560221404270414139663128 </id> <phrase> loss </phrase> <category> ['effacement'] </category> ),
 (<id> 216352204347299563531621281619235742488 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 216377669071013273232367222580313236248 </id> <phrase> white matter </phrase> <category> ['brain_anatomy'] </category> ),
 (<id> 216352204347299563531621281619235742488 </id> <phrase> sulc </phrase> <category> ['sulci'] </category> ,
  <id> 216377191325193312218411533510292710168 </id> <phrase> loss </phrase> <category> ['effacement'] </category> ),
 (<id> 216352204347299563531621281619235742488 </id> <phrase>

In [10]:
mtf.confirm_existence("effacement","categoryString")

2

In [11]:
#if mtf.find_mod_target_pair("effacement","getLiteral","sulc","getLiteral"):
if mtf.find_mod_target_pair("efface","getLiteral","sulc","getLiteral") and mtf.confirm_existence("effacement","categoryString"):
    print(True)

In [12]:
import re
midline_shift_cm_pattern = re.compile('[0-9]+\.*[0-9]* cm')
cm_search = midline_shift_cm_pattern.search("There is increase associated regional mass effect with midline shift to the right measuring approximately 1.4 cm, previously this was minimal if present in addition there is no effacement of the left lateral ventricle")
int(float(cm_search.group().split(" ")[0])*10)
#midline_shift = int(float(cm_search.group().split(" ")[0])*10)

14

In [13]:
# import re
# class edema_extractor:
#     def __init__(self, reports):
#         self.reports=reports
        
#         #Setting up pyConText
#         self.mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/modifiers_fu.yml"
#         self.targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/targets_fu.yml"
#         self.itemData_init = pyConText_itemData_initializer(self.mod_path,self.targ_path)
#         self.modifiers = self.itemData_init.get_itemData()[0]
#         self.targets = self.itemData_init.get_itemData()[1]
#         self.midline_shift_volume_pattern = re.compile('[0-9]+ mm')
#         #Currently cm is flagged with -1, future work could involve processing cm values
# #         self.midline_shift_cm_pattern = re.compile('[0-9]+\.*[0-9]* cm')
        
#         #Outputs
#         self.e_ms_se = []
#         self.edema = []
#         self.midline_shift = []
#         self.sulcal_effacement = []
        
#         for row in self.reports.itertuples():
#             r1 = row.fu_CT1_report
#             r2 = row.fu_CT2_report
#             r3 = row.fu_CT3_report
#             r4 = row.fu_CT4_report
#             row_reports = [r1,r2,r3,r4]
#             raw_e_ms_se_list = [self.extract_edema(r) for r in row_reports]
#             e_ms_se_list = []
#             for tup in raw_e_ms_se_list:
#                 if not None in tup:
#                     e_ms_se_list.append(tup)
            
#             self.e_ms_se.append(e_ms_se_list)
            
#             #No report
#             if e_ms_se_list == []:
#                 self.edema.append(None)
#                 self.midline_shift.append(None)
#                 self.sulcal_effacement.append(None)
#             else:
#                 edema_list = [tup[0] for tup in e_ms_se_list]
#                 ms_list = [tup[1] for tup in e_ms_se_list]
#                 se_list = [tup[2] for tup in e_ms_se_list]
#                 self.edema.append(max(edema_list))
#                 self.midline_shift.append(max(ms_list))
#                 self.sulcal_effacement.append(max(se_list))
            
            
            
# #             if str(report) != "nan":
# #                 self.extract_edema(report)
# #             else:
# #                 self.edema.append(None)
# #                 self.midline_shift.append(None)
# #                 self.sulcal_effacement.append(None)
                
#     def get_edema(self):
#         return self.edema
    
#     def get_midline_shift(self):
#         return self.midline_shift
    
#     def get_sulcal_effacement(self):
#         return self.sulcal_effacement
    
#     def get_raw(self):
#         return self.e_ms_se
                
                    
                    
                    
#     #Returns edema=0/1
#     #Returns midline_shift=0/1
#     #Returns sulcal_effacement=0/1
    
#     def extract_edema(self,report):
#         edema = None
#         midline_shift = None
#         sulcal_effacement = None
#         #Report is nan
#         if type(report) != str:
#             return edema, midline_shift, sulcal_effacement

#         sents = report.split(".")
        
#         for sent in sents:
#             ms = self.is_midline_shift(sent)
#             ed = self.is_edema(sent)
#             se = self.is_sulcal_effacement(sent)
          
#             if ms is not None:
#                 if ms > 0:
#                     search = self.midline_shift_volume_pattern.search(sent)
#                     if search:
#                         #Get '3' from 'there is 3 mm rightward midline shift'
#                         midline_shift = int(search.group().split(" ")[0])
#                     else:
# #                         cm_search = self.midline_shift_cm_pattern.search(sent)
# #                         if cm_search:
# #                             print(cm_search.group().split(" ")[0])
# #                             midline_shift = int(float(cm_search.group().split(" ")[0])*10)
#                         #Debugging value for midline shift without volume estimate
#                         midline_shift = -1
#                 else:
#                     midline_shift = 0
            
#             if ed is not None:
#                 if ed > 0:
#                     edema = 1
#                 else:
#                     edema = 0
            
#             if se is not None:
#                 if se > 0:
#                     sulcal_effacement = 1
#                 else:
#                     sulcal_effacement = 0
                
#             #print("Edema "+str(edema)+" midline_shift "+str(midline_shift))
#             #Greedy algorithm, will only read sentences until it has answer for all 3
#             if midline_shift is not None and edema is not None and sulcal_effacement is not None:
#                 break
        
#         #Didn't find midline shift
#         if midline_shift is None:
#             midline_shift = 0
#         #Didn't find edema
#         if edema is None:
#             edema = 0
#         #Didn't find sulcal effacement
#         if sulcal_effacement is None:
#             sulcal_effacement = 0
            
#         #In erroneous case of midline shift without edema
#         if midline_shift > 1 and edema == 0:
#             edema = 1
            
#         return edema, midline_shift, sulcal_effacement
            
# #         self.midline_shift.append(midline_shift)
# #         self.edema.append(edema)
# #         self.sulcal_effacement.append(sulcal_effacement)
        
        
        
        
#     def is_midline_shift(self,sent):
#         if "midline shift" in sent.lower():
#             mtf = modifier_target_finder(sent,self.modifiers,self.targets)
#             #0 for negation
#             #1 for present in sent
#             #2 for declared existence
#             return mtf.confirm_existence("midline_shift","categoryString")
#         return None
     
#     def is_edema(self,sent):
#         #Check for "edema" first
#         if "edema" in sent.lower():
#             mtf = modifier_target_finder(sent,self.modifiers,self.targets)
#             #0 for negation
#             #1 for present in sent
#             #2 for declared existence
#             return mtf.confirm_existence("edema","categoryString")
#         return None
    
#     def is_sulcal_effacement(self,sent):
#         #Edema not found, check for 'effacement of sulci' or 'sulcal effacement' or loss of sulci
#         if "sulc" in sent.lower():
#             mtf = modifier_target_finder(sent,self.modifiers,self.targets)
#             if mtf.find_mod_target_pair("effacement","categoryString","sulci","categoryString"):
#                 #0 for negation
#                 #1 for present in sent
#                 #2 for declared existence
#                 return mtf.confirm_existence("effacement","categoryString") 
            
#         return None
    
            
        
        
                    
                
            

In [14]:
import re
class edema_extractor:
    def __init__(self, reports):
        self.reports=reports
        
        #Setting up pyConText
        self.mod_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/modifiers_fu.yml"
        self.targ_path = "/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/targets_fu.yml"
        self.itemData_init = pyConText_itemData_initializer(self.mod_path,self.targ_path)
        self.modifiers = self.itemData_init.get_itemData()[0]
        self.targets = self.itemData_init.get_itemData()[1]
        self.midline_shift_volume_pattern = re.compile('[0-9]+ mm')
        #Currently cm is flagged with -1, future work could involve processing cm values
        self.midline_shift_cm_pattern = re.compile('[0-9]+\.*[0-9]* cm')
        
        #Outputs
        self.e_ms_se = []
        self.edema = []
        self.midline_shift = []
        self.sulcal_effacement = []
        self.ms_lists = []
        
        for row in self.reports.itertuples():
            r1 = row.fu_CT1_report
            r2 = row.fu_CT2_report
            r3 = row.fu_CT3_report
            r4 = row.fu_CT4_report
            row_reports = [r1,r2,r3,r4]
            raw_e_ms_se_list = [self.extract_edema(r) for r in row_reports]
            e_ms_se_list = []
            for tup in raw_e_ms_se_list:
                if not None in tup:
                    e_ms_se_list.append(tup)
            
            self.e_ms_se.append(e_ms_se_list)
            
            #No report
            if e_ms_se_list == []:
                self.edema.append(None)
                self.midline_shift.append(None)
                self.sulcal_effacement.append(None)
                self.ms_lists.append(None)
            else:
                edema_list = [tup[0] for tup in e_ms_se_list]
                ms_list = [tup[1] for tup in e_ms_se_list]
                se_list = [tup[2] for tup in e_ms_se_list]
                self.edema.append(max(edema_list))
                self.midline_shift.append(max(ms_list))
                self.sulcal_effacement.append(max(se_list))
                self.ms_lists.append(ms_list)
            
            
            
#             if str(report) != "nan":
#                 self.extract_edema(report)
#             else:
#                 self.edema.append(None)
#                 self.midline_shift.append(None)
#                 self.sulcal_effacement.append(None)
                
    def get_edema(self):
        return self.edema
    
    def get_midline_shift(self):
        return self.midline_shift
    
    def get_ms_lists(self):
        return self.ms_lists 
    
    def get_sulcal_effacement(self):
        return self.sulcal_effacement
    
    def get_raw(self):
        return self.e_ms_se
                
                    
                    
                    
    #Returns edema=0/1
    #Returns midline_shift=0/1
    #Returns sulcal_effacement=0/1
    
    def extract_edema(self,report):
        edema = None
        midline_shift = None
        sulcal_effacement = None
        #Report is nan
        if type(report) != str:
            return edema, midline_shift, sulcal_effacement

#         sents = report.split(".")
        
        ms = self.is_midline_shift(report)
        ed = self.is_edema(report)
        se = self.is_sulcal_effacement(report)
        
#         for sent in sents:
#             ms = self.is_midline_shift(sent)
#             ed = self.is_edema(sent)
#             se = self.is_sulcal_effacement(sent)


          
        if ms is not None:
            if ms > 0:
                search = self.midline_shift_volume_pattern.search(report)
                if search:
                    #Get '3' from 'there is 3 mm rightward midline shift'
                    midline_shift = int(search.group().split(" ")[0])
                else:
                    cm_search = self.midline_shift_cm_pattern.search(report)
                    if cm_search:
                        print(cm_search.group().split(" ")[0])
                        midline_shift = int(float(cm_search.group().split(" ")[0])*10)
                    #Debugging value for midline shift without volume estimate
                    #midline_shift = -1
            else:
                midline_shift = 0

        if ed is not None:
            if ed > 0:
                edema = 1
            else:
                edema = 0

        if se is not None:
            if se > 0:
                sulcal_effacement = 1
            else:
                sulcal_effacement = 0
                
            #print("Edema "+str(edema)+" midline_shift "+str(midline_shift))
            #Greedy algorithm, will only read sentences until it has answer for all 3
#             if midline_shift is not None and edema is not None and sulcal_effacement is not None:
#                 break
        
        #Didn't find midline shift
        if midline_shift is None:
            midline_shift = 0
        #Didn't find edema
        if edema is None:
            edema = 0
        #Didn't find sulcal effacement
        if sulcal_effacement is None:
            sulcal_effacement = 0
            
        #In erroneous case of midline shift without edema
        if midline_shift > 1 and edema == 0:
            edema = 1
            
        return edema, midline_shift, sulcal_effacement
            
#         self.midline_shift.append(midline_shift)
#         self.edema.append(edema)
#         self.sulcal_effacement.append(sulcal_effacement)
        
        
        
        
    def is_midline_shift(self,sent):
        if "midline shift" in sent.lower():
            mtf = modifier_target_finder(sent,self.modifiers,self.targets)
            #0 for negation
            #1 for present in sent
            #2 for declared existence
            return mtf.confirm_existence("midline_shift","categoryString")
        return None
     
    def is_edema(self,sent):
        #Check for "edema" first
        if "edema" in sent.lower():
            mtf = modifier_target_finder(sent,self.modifiers,self.targets)
            #0 for negation
            #1 for present in sent
            #2 for declared existence
            return mtf.confirm_existence("edema","categoryString")
        return None
    
    def is_sulcal_effacement(self,sent):
        #Edema not found, check for 'effacement of sulci' or 'sulcal effacement' or loss of sulci
        if "sulc" in sent.lower():
            mtf = modifier_target_finder(sent,self.modifiers,self.targets)
            if mtf.find_mod_target_pair("effacement","categoryString","sulci","categoryString"):
                #0 for negation
                #1 for present in sent
                #2 for declared existence
                return mtf.confirm_existence("effacement","categoryString") 
            
        return None
    
            
        
        
                    
                
            

In [15]:
i=0
ee = edema_extractor(reports)
for row in reports.itertuples():
    r1 = row.fu_CT1_report
    r2 = row.fu_CT2_report
    r3 = row.fu_CT3_report
    r4 = row.fu_CT4_report
    row_reports = [r1,r2,r3,r4]
    ee_list = [ee.extract_edema(r) for r in row_reports]
    i+=1
    print(ee_list)
    if i == 2:
        break
new_ee = []
for tup in ee_list:
    if not None in tup:
        new_ee.append(tup)


1.6
1
1.1
[(None, None, None), (None, None, None), (None, None, None), (None, None, None)]
[(1, 0, 0), (1, 0, 1), (1, 4, 0), (None, None, None)]


In [16]:
ee_list
new_ee
edema_list = [tup[1] for tup in new_ee]
edema_list
max(edema_list)

4

In [17]:
reports["fu_CT1_report"][1]


'New hematoma centered at the right insula and likely right caudate with extensive surrounding cerebral edema of the right cerebral hemisphere and mass effect upon the right lateral ventricle which is effaced.'

In [18]:
# ee_test = edema_extractor([reports["fu_CT_report"][2]])
# ee_test.edema


In [19]:
ee_test = edema_extractor(reports)
ee_test.edema

1.6
1
1.1


[None,
 1,
 0,
 None,
 None,
 1,
 1,
 0,
 1,
 0,
 None,
 1,
 1,
 1,
 0,
 None,
 1,
 0,
 0,
 1,
 None,
 None,
 None,
 None,
 0,
 0,
 1,
 1,
 1,
 None,
 0,
 None,
 0,
 1,
 None,
 0,
 0,
 0,
 None,
 1,
 0,
 0,
 0,
 0,
 1,
 None,
 None,
 1,
 1,
 0,
 0,
 None,
 0,
 0,
 0,
 1,
 None,
 None,
 0,
 None,
 None,
 0,
 1,
 1,
 None,
 None,
 None,
 1,
 0,
 0,
 None,
 None,
 None,
 0,
 0,
 None,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 None,
 0,
 None,
 0,
 0,
 None,
 None,
 None,
 None,
 None,
 0,
 1,
 1,
 None]

In [20]:
ee_test.midline_shift

[None,
 4,
 0,
 None,
 None,
 16,
 0,
 0,
 0,
 0,
 None,
 0,
 0,
 10,
 0,
 None,
 11,
 0,
 0,
 0,
 None,
 None,
 None,
 None,
 0,
 0,
 0,
 3,
 0,
 None,
 0,
 None,
 0,
 3,
 None,
 0,
 0,
 0,
 None,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 None,
 9,
 5,
 0,
 0,
 None,
 0,
 0,
 0,
 0,
 None,
 None,
 0,
 None,
 None,
 0,
 6,
 13,
 None,
 None,
 None,
 9,
 0,
 0,
 None,
 None,
 None,
 0,
 0,
 None,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 None,
 0,
 None,
 0,
 0,
 None,
 None,
 None,
 None,
 None,
 0,
 2,
 0,
 None]

In [21]:
ee = edema_extractor(reports)
ee.edema[:5]

1.6
1
1.1


[None, 1, 0, None, None]

In [22]:
ee.midline_shift[:5]

[None, 4, 0, None, None]

In [23]:
ee.sulcal_effacement[:5]

[None, 1, 0, None, None]

In [24]:
edema_df = raw_df
edema_df["edema"] = ee.get_edema()
edema_df["midline shift (mm)"] = ee.get_midline_shift()
edema_df["sulcal effacement"] = ee.get_sulcal_effacement()
edema_df["ee_ms_se"] = ee.get_raw()
edema_df["ms_lists"] = ee.get_ms_lists()

In [25]:
# #Malignant edema derived from Deterioration ("ME" or "herniation"), DHC, Osmotics
# edema_df["ME"] = 0
# edema_df["DHC"].fillna(0,inplace=True)
# edema_df["Osmotics"].fillna(0,inplace=True)
# # edema_df["Deterioration"].fillna("",inplace=True)
# edema_df["DHC"].astype("int")
# edema_df["Osmotics"].astype("int")
# # edema_df["Osmotics"].astype("str")

# edema_df.loc[edema_df["DHC"]>0,"ME"] = 1
# edema_df.loc[edema_df["Osmotics"]>0,"ME"] = 1
# edema_df.loc[edema_df["Deterioration"]>0,"ME"] = 1

# edema_df[edema_df["ME"] > 0] 

In [26]:
edema_df["ME"] = edema_df["Malignant Edema"]
edema_df[edema_df["ME"] > 0] 

,Organization,Point of Care,Accession Number,Modality,Exam Code,Exam Description,CPT Code,Report Text,duplicate,LVO,...,Comments.1,Billing,manual_edema,manual_mls,edema,midline shift (mm),sulcal effacement,ee_ms_se,ms_lists,ME
9,BJH,BJH I104,55254511,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,NaN,1.100159e+11,1.0,16.0,1.0,16.0,1.0,"[(1, 5, 1), (1, 16, 0)]","[5, 16]",1.0
12,BJH,BJH ED,55211821,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,midline shift and herniation noted after DHC,1.100158e+11,1.0,12.0,1.0,0.0,1.0,"[(0, 0, 1), (0, 0, 0), (0, 0, 0), (1, 0, 0)]","[0, 0, 0, 0]",1.0
21,BJH,BJH 94ICU,54931869,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,no DHC offered due to large left sided stroke ...,1.100154e+11,1.0,10.0,1.0,10.0,0.0,"[(1, 10, 0)]",[10],1.0
25,BJH,BJH ED,54827019,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION:_x000D_\nComputed tomography angio...,NaN,1,...,no DHC offered - late development of edema 10 ...,1.100153e+11,1.0,14.0,1.0,11.0,0.0,"[(0, 0, 0), (1, 11, 0)]","[0, 11]",1.0
40,BJH,BJH 94ICU,54504585,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,one dose of 23.4%. Death due to WOC from poor ...,1.100149e+11,1.0,3.0,1.0,3.0,0.0,"[(1, 0, 0), (1, 3, 0), (1, 3, 0)]","[0, 3, 3]",1.0
49,BJH,BJH 0105,54353306,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,"WOC, no DHC offered due to poor premorbid sta...",1.100147e+11,NaN,NaN,0.0,0.0,0.0,"[(0, 0, 0), (0, 0, 0)]","[0, 0]",1.0
53,BJH,BJH ED,54265071,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,comfort care,1.100146e+11,1.0,14.0,1.0,0.0,0.0,"[(1, 0, 0)]",[0],1.0
63,BJH,BJH I104,54088786,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,"family declined DHC and made patient DNR, comf...",1.100144e+11,1.0,10.0,1.0,5.0,0.0,"[(1, 0, 0), (1, 5, 0), (1, 5, 0), (1, 5, 0)]","[0, 5, 5, 5]",1.0
79,BJH,BJH 94ICU,55372364,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,TRISL,1.100160e+11,1.0,11.0,1.0,6.0,0.0,"[(1, 5, 0), (1, 6, 0)]","[5, 6]",1.0
80,BJH,BJH 94ICU,55361527,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: _x000D_\nComputed tomography angi...,NaN,1,...,Brain death,1.100160e+11,1.0,13.0,1.0,13.0,0.0,"[(1, 11, 0), (1, 13, 0), (1, 0, 0)]","[11, 13, 0]",1.0


In [27]:
edema_df["manual_edema"].fillna(0,inplace=True)
edema_df["edema"].fillna(0,inplace=True)
edema_df["midline shift (mm)"].fillna(0,inplace=True)
edema_df["manual_mls"].fillna(0,inplace=True)
edema_df

,Organization,Point of Care,Accession Number,Modality,Exam Code,Exam Description,CPT Code,Report Text,duplicate,LVO,...,Comments.1,Billing,manual_edema,manual_mls,edema,midline shift (mm),sulcal effacement,ee_ms_se,ms_lists,ME
0,BJH,BJH 94ICU,55426263,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,no f/u CT scans,1.100161e+11,0.0,0.0,0.0,0.0,NaN,[],None,0.0
1,BJH,BJH I104,55391713,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,NaN,1.100160e+11,1.0,4.0,1.0,4.0,1.0,"[(1, 0, 0), (1, 0, 1), (1, 4, 0)]","[0, 0, 4]",0.0
5,BJH,BJH ED,55320338,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,NaN,1.100159e+11,0.0,0.0,0.0,0.0,0.0,"[(0, 0, 0)]",[0],0.0
6,BJH,BJH 0145,55307030,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,no f/u CT scans,1.100159e+11,0.0,0.0,0.0,0.0,NaN,[],None,0.0
7,BJH,BJH ED,55297602,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,no f/u CT scans,1.100159e+11,0.0,0.0,0.0,0.0,NaN,[],None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,BJH,BJH I104,53953898,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,no f/u CT scans,1.100142e+11,0.0,0.0,0.0,0.0,NaN,[],None,0.0
115,BJH,BJH 94ICU,53909015,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,BJH_027,1,...,NaN,1.000246e+11,0.0,0.0,0.0,0.0,0.0,"[(0, 0, 0)]",[0],0.0
116,BJH,BJH I104,53898972,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: Computed tomography angiography (...,NaN,1,...,NaN,1.100141e+11,1.0,2.0,1.0,2.0,1.0,"[(0, 0, 1), (0, 0, 0), (1, 2, 0)]","[0, 0, 2]",0.0
117,BJH,BJH I104,53880568,CTA,IMG575,CTA/CTP RAPID STROKE (C),70496,EXAMINATION: _x000D_\nComputed tomography angi...,NaN,1,...,NaN,1.100141e+11,1.0,0.0,1.0,0.0,1.0,"[(1, 0, 1)]",[0],0.0


In [28]:
from sklearn.metrics import classification_report
print(classification_report(edema_df["manual_edema"], edema_df["edema"]))

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99        69
         1.0       1.00      0.93      0.96        28

    accuracy                           0.98        97
   macro avg       0.99      0.96      0.97        97
weighted avg       0.98      0.98      0.98        97



In [29]:
mls = edema_df[edema_df["manual_mls"]>0]
print(classification_report(mls["manual_mls"], mls["midline shift (mm)"]))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         2.0       1.00      1.00      1.00         2
         3.0       1.00      0.67      0.80         3
         4.0       1.00      1.00      1.00         1
         5.0       0.00      0.00      0.00         0
         6.0       0.00      0.00      0.00         0
         9.0       1.00      1.00      1.00         2
        10.0       1.00      0.50      0.67         2
        11.0       0.00      0.00      0.00         1
        12.0       0.00      0.00      0.00         1
        13.0       1.00      1.00      1.00         1
        14.0       0.00      0.00      0.00         2
        16.0       1.00      1.00      1.00         1
        50.0       0.00      0.00      0.00         1

    accuracy                           0.59        17
   macro avg       0.50      0.44      0.46        17
weighted avg       0.71      0.59      0.63        17



/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
#PRESENCE OF MLS
mls.loc[mls["manual_mls"]>0] = 1
mls.loc[mls["midline shift (mm)"]>0] = 1
print(classification_report(mls["manual_mls"], mls["midline shift (mm)"]))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        17

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/mnt/c/Users/zohai/Documents/Research/Neuro_ML/miniconda3/envs/neuroml/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [31]:
output_filename = filename.split(".")[0]
output_filename += "_AUTOMATED_EDEMA_RESULTS.xlsx"
output_filename

'/mnt/c/Users/zohai/Documents/Research/Neuro_ML/LVOs_Follow_Up_CTs/LVO_CTP_19_20_occlusions_with_edema_AUTOMATED_EDEMA_RESULTS.xlsx'

In [32]:
edema_df.to_excel(output_filename, index=False)